## 한국수출입은행 - 원화 환율(최근까지) 및 DB 저장
+ 작성: 임경호
+ 한국수출입은행 Open API https://www.koreaexim.go.kr/ir/HPHKIR019M01

### 1. 데이터를 가져올 날짜 설정
+ 저장된 데이터 마지막 일자의 다음날부터 현재일 기준 직전일까지 데이터를 일괄로 가져와서 저장

In [1]:
import requests
import pandas as pd

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect
import myutil

conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

query = f'SELECT date FROM fx_eximbank ORDER BY date DESC LIMIT 1'
cur.execute(query)

row = cur.fetchone()
last_date = row[0]     # 저장된 데이터의 마지막 일자

conn.close()

date_from = myutil.get_next_day(last_date)
date_to = myutil.get_previous_day()

In [2]:
print(date_from, date_to)

20230908 20230910


### 2. API를 활용한 데이터 수집 (한국수출입은행 Open API)

In [3]:
url = 'https://www.koreaexim.go.kr/site/program/financial/exchangeJSON'
api_key = 'OczbdUJk31QuSPZNx3llq7VFOEmsMuew'
RESULT_SUCCESS = 1

date_range = pd.date_range(start=date_from, end=date_to, freq="D")

df_fx_all = pd.DataFrame()
for dt in date_range:
    date_str = dt.strftime("%Y%m%d")
    params = {
        'authkey': api_key,
        'searchdate': date_str,
        'data': 'AP01'             # 환율
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:     # URL GET '200 정상'
        print("URL GET Error")
        pass
    df_fx = pd.json_normalize(response.json())
    if not df_fx.empty:
        # print(df_fx['result'].unique())
        if RESULT_SUCCESS not in df_fx['result'].unique():         # 성공이 아니라면
            print(f'{date_str}: Result is not successful')
            pass
        df_fx.drop('result', axis=1, inplace=True)
        df_fx.insert(0, 'date', date_str)
        df_fx_all = pd.concat([df_fx_all, df_fx], ignore_index=True)

In [4]:
df_fx_all.shape

(23, 11)

In [5]:
df_fx_all

,date,cur_unit,ttb,tts,deal_bas_r,bkpr,yy_efee_r,ten_dd_efee_r,kftc_bkpr,kftc_deal_bas_r,cur_nm
0,20230908,AED,359.75,367.02,363.39,363,0,0,363,363.39,아랍에미리트 디르함
1,20230908,AUD,842.43,859.44,850.94,850,0,0,850,850.94,호주 달러
2,20230908,BHD,"3,505","3,575.81","3,540.41","3,540",0,0,"3,540","3,540.41",바레인 디나르
3,20230908,BND,967.41,986.96,977.19,977,0,0,977,977.19,브루나이 달러
4,20230908,CAD,965.47,984.98,975.23,975,0,0,975,975.23,캐나다 달러
5,20230908,CHF,"1,480.08","1,509.99","1,495.04","1,495",0,0,"1,495","1,495.04",스위스 프랑
6,20230908,CNH,180.19,183.84,182.02,182,0,0,182,182.02,위안화
7,20230908,DKK,189.46,193.29,191.38,191,0,0,191,191.38,덴마아크 크로네
8,20230908,EUR,"1,413.32","1,441.87","1,427.6","1,427",0,0,"1,427","1,427.6",유로
9,20230908,GBP,"1,647.65","1,680.94","1,664.3","1,664",0,0,"1,664","1,664.3",영국 파운드


### 3. 데이터 전처리

In [6]:
df_fx_all_save = df_fx_all[['date', 'cur_unit', 'ttb', 'tts', 'deal_bas_r', 'bkpr', 'kftc_bkpr', 'kftc_deal_bas_r']]

In [ ]:
# 특정 통화 환율 확인
# df_fx_all_save[df_fx_all_save['cur_unit'].str.contains('USD')]

In [7]:
# 숫자의 소수점(,) 없애기
df_fx_all_save.iloc[:,2:8] = df_fx_all_save.iloc[:,2:8].apply(lambda x: x.str.replace(',', ''), axis = 1)
# 통화코드에서 세 자리 초과하는 내용은 지우고 통화코드 생성: (예) JPY(100) -> JPY로 변경
df_fx_all_save.insert(2, 'cur_code', df_fx_all_save['cur_unit'].str[0:3])
df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [8]:
df_fx_all_save

,date,cur_code,ttb,tts,deal_bas_r,bkpr,kftc_bkpr,kftc_deal_bas_r
0,20230908,AED,359.75,367.02,363.39,363,363,363.39
1,20230908,AUD,842.43,859.44,850.94,850,850,850.94
2,20230908,BHD,3505,3575.81,3540.41,3540,3540,3540.41
3,20230908,BND,967.41,986.96,977.19,977,977,977.19
4,20230908,CAD,965.47,984.98,975.23,975,975,975.23
5,20230908,CHF,1480.08,1509.99,1495.04,1495,1495,1495.04
6,20230908,CNH,180.19,183.84,182.02,182,182,182.02
7,20230908,DKK,189.46,193.29,191.38,191,191,191.38
8,20230908,EUR,1413.32,1441.87,1427.6,1427,1427,1427.6
9,20230908,GBP,1647.65,1680.94,1664.3,1664,1664,1664.3


In [ ]:
# 특정 통화 환율 확인
# df_fx_all_save[df_fx_all_save['cur_code'].str.contains('USD')]

### 4. 테이블에 데이터 저장 (mdw.weather)

In [9]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_fx_all_save.itertuples():
    sql = "insert into fx_eximbank (date, cur_code, ttb, tts, deal_bas_r, bkpr, kftc_bkpr, kftc_deal_bas_r) \
                           values (%s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8]))
conn.commit()
conn.close()